In [2]:
import numpy as np
from pycalphad import Database, Model, variables as v
from pycalphad.tests.datasets import ALFE_TDB

dbf = Database(ALFE_TDB)
mod_liq = Model(dbf, ['AL', 'FE'], 'LIQUID')
save = mod_liq.models['idmix']
mod_liq.models.clear()
mod_liq.models['idmix'] = save
mod_alfe = Model(dbf, ['AL', 'FE', 'VA'], 'AL13FE4')

In [3]:
from pycalphad.core.problem import Problem
from pycalphad.core.composition_set import CompositionSet
from pycalphad.codegen.callables import build_phase_records

conds = {v.N: 1, v.P: 1e5, v.T: 300, v.X('AL'): 0.8}
comps = [v.Species('AL'), v.Species('FE'), v.Species('VA')]

prx = build_phase_records(dbf, comps, ['LIQUID', 'AL13FE4'], conds,
                         models={'LIQUID': mod_liq, 'AL13FE4': mod_alfe}, build_gradients=True, build_hessians=True)
cs_liq = CompositionSet(prx['LIQUID'])
cs_alfe = CompositionSet(prx['AL13FE4'])
#cs_liq.update(np.array([1.00000000e+00,3.67650217e-13, 1.00000000e+00]), 1.48962701e-01,
#              np.array([1.00000000e+00, 1.00000000e+05, 3.00000000e+02]), True)
#cs_alfe.update(np.array([1.00000000e+00, 1.00000000e+00, 9.99773278e-01, 2.26721529e-04]), 8.51037299e-01,
#               np.array([1.00000000e+00, 1.00000000e+05, 3.00000000e+02]), True)
prob = Problem([cs_liq, cs_alfe], comps, {str(key): value for key, value in conds.items()})

In [4]:
x = [1.00000000e+00, 1.00000000e+05, 3.00000000e+02, 1.00000000e+00,
       6.52975359e-13, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       9.99773278e-01, 2.26721528e-04, 1.48962701e-01, 8.51037299e-01]
x_exact = [1.00000000e+00, 1.00000000e+05, 3.00000000e+02, 1.00000000e+00,
           3.67650217e-13, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
           9.99773278e-01, 2.26721529e-04, 1.48962701e-01, 8.51037299e-01]
x = x_exact

In [5]:
from pycalphad import equilibrium, variables as v
equilibrium(dbf, ['AL', 'FE', 'VA'], ['LIQUID'], {v.P: 1e5, v.T: 300, v.X('AL'): 0.3}, model={'LIQUID':mod_liq}).MU

<xarray.DataArray 'MU' (N: 1, P: 1, T: 1, X_AL: 1, component: 2)>
array([[[[[-3003.12956447,  -889.67214641]]]]])
Coordinates:
  * N          (N) float64 1.0
  * P          (P) float64 1e+05
  * T          (T) float64 300.0
  * X_AL       (X_AL) float64 0.3
  * component  (component) <U2 'AL' 'FE'

In [9]:
import numpy as np
num_statevars = 3
num_components = 2
#chemical_potentials = np.array([-8490.7, -123110])
chemical_potentials = np.array([-3003.12956447,  -889.67214641])
delta_statevars = np.zeros(num_statevars)
#phase_amt = np.array([1.48962701e-01, 8.51037299e-01])
phase_amt = np.array([1.0])
dof = [None]#, None]
# Exact solution
#dof[0] = np.array([1.00000000e+00, 1.00000000e+05, 3.00000000e+02, 1.00000000e+00, 6.52975359e-13])
#dof[1] = np.array([1.00000000e+00, 1.00000000e+05, 3.00000000e+02, 1.00000000e+00,
#                   1.00000000e+00, 9.99773278e-01, 2.26721529e-04])
# Tests
dof[0] = np.array([1.00000000e+00, 1.00000000e+05, 3.00000000e+02, 0.3, 0.7])
#dof[0] = np.array([1.00000000e+00, 1.00000000e+05, 3.00000000e+02, 0.5, 0.5])
#dof[1] = np.array([1.00000000e+00, 1.00000000e+05, 3.00000000e+02, 1.00000000e+00,
#                   1.00000000e+00, 9.99773278e-01, 2.26721529e-04])

for iteration in range(10):
    true_delta_y = [None, None]
    for inner_iter in range(10):
        for idx, compset in enumerate((cs_liq, )):
            # TODO: Activate bound constraints
            # TODO: Use better dof storage
            x = dof[idx]
            # Compute phase matrix (LHS of Eq. 41, Sundman 2015)
            phase_matrix = np.zeros((compset.phase_record.phase_dof + compset.phase_record.num_internal_cons,
                                     compset.phase_record.phase_dof + compset.phase_record.num_internal_cons))
            hess_tmp = np.zeros((num_statevars + compset.phase_record.phase_dof,
                                num_statevars + compset.phase_record.phase_dof))
            cons_jac_tmp = np.zeros((compset.phase_record.num_internal_cons,
                                     num_statevars + compset.phase_record.phase_dof))
            compset.phase_record.hess(hess_tmp, x)
            phase_matrix[:compset.phase_record.phase_dof, :compset.phase_record.phase_dof] = hess_tmp[num_statevars:, num_statevars:]
            compset.phase_record.internal_cons_jac(cons_jac_tmp, x)
            phase_matrix[compset.phase_record.phase_dof:, :compset.phase_record.phase_dof] = cons_jac_tmp[:, num_statevars:]
            phase_matrix[:compset.phase_record.phase_dof, compset.phase_record.phase_dof:] = cons_jac_tmp[:, num_statevars:].T
            #print(phase_matrix)

            # Compute right-hand side of Eq. 41, Sundman 2015
            rhs = np.zeros(compset.phase_record.phase_dof + compset.phase_record.num_internal_cons)
            grad_tmp = np.zeros(num_statevars + compset.phase_record.phase_dof)
            compset.phase_record.grad(grad_tmp, x)
            rhs[:compset.phase_record.phase_dof] = -grad_tmp[num_statevars:]
            rhs[:compset.phase_record.phase_dof] -= np.dot(hess_tmp[num_statevars:,:num_statevars], delta_statevars)
            mass_jac_tmp = np.zeros((num_components, num_statevars + compset.phase_record.phase_dof))
            for comp_idx in range(num_components):
                compset.phase_record.mass_grad(mass_jac_tmp[comp_idx, :], x, comp_idx)
            # Q: Do I need to multiply the mass gradient by the phase_amt?
            rhs[:compset.phase_record.phase_dof] += mass_jac_tmp.T.dot(chemical_potentials)[num_statevars:]
            #print(rhs)
            soln = np.linalg.solve(phase_matrix, rhs)
            delta_y = soln[:compset.phase_record.phase_dof]
            old_y = np.array(x[num_statevars:])
            new_y = old_y + delta_y
            new_y[new_y < 1e-15] = 1e-15
            new_y[new_y > 1] = 1
            x[num_statevars:] = new_y
            true_delta_y[idx] = new_y - old_y
            print(compset.phase_record.phase_name, idx, true_delta_y[idx])
    # TODO: Update state variables
    num_stable_phases = 1
    equilibrium_matrix = np.zeros((num_stable_phases + num_components, num_stable_phases + num_components))
    equilibrium_rhs = np.zeros(num_stable_phases + num_components)
    for idx, compset in enumerate((cs_liq, )):
        # TODO: Activate bound constraints
        # TODO: Use better dof storage
        x = dof[idx]
        energy_tmp = np.zeros((1,1))
        compset.phase_record.obj(energy_tmp[:,0], x)
        equilibrium_rhs[idx] = energy_tmp[0,0]
        masses_tmp = np.zeros((num_components,1))
        mass_jac_tmp = np.zeros((num_components, num_statevars + compset.phase_record.phase_dof))
        for comp_idx in range(num_components):
            compset.phase_record.mass_grad(mass_jac_tmp[comp_idx, :], x, comp_idx)
            compset.phase_record.mass_obj(masses_tmp[comp_idx, :], x, comp_idx)
        # Compute phase matrix (LHS of Eq. 41, Sundman 2015)
        phase_matrix = np.zeros((compset.phase_record.phase_dof + compset.phase_record.num_internal_cons,
                                 compset.phase_record.phase_dof + compset.phase_record.num_internal_cons))
        hess_tmp = np.zeros((num_statevars + compset.phase_record.phase_dof,
                            num_statevars + compset.phase_record.phase_dof))
        cons_jac_tmp = np.zeros((compset.phase_record.num_internal_cons,
                                 num_statevars + compset.phase_record.phase_dof))
        compset.phase_record.hess(hess_tmp, x)
        phase_matrix[:compset.phase_record.phase_dof, :compset.phase_record.phase_dof] = hess_tmp[num_statevars:, num_statevars:]
        compset.phase_record.internal_cons_jac(cons_jac_tmp, x)
        phase_matrix[compset.phase_record.phase_dof:, :compset.phase_record.phase_dof] = cons_jac_tmp[:, num_statevars:]
        phase_matrix[:compset.phase_record.phase_dof, compset.phase_record.phase_dof:] = cons_jac_tmp[:, num_statevars:].T
        e_matrix = np.linalg.inv(phase_matrix)[:compset.phase_record.phase_dof, :compset.phase_record.phase_dof]
        # Eq. 44
        c_G = -np.dot(e_matrix, grad_tmp[num_statevars:])
        c_T = None # TODO
        c_component = np.dot(mass_jac_tmp[:, num_statevars:], e_matrix)
        equilibrium_matrix[idx, :num_components] = masses_tmp[:, 0]
        equilibrium_matrix[num_stable_phases:, num_components+idx] = masses_tmp[:, 0]
        for comp_idx_1 in range(num_components):
            equilibrium_rhs[num_stable_phases+comp_idx_1] += phase_amt[idx] * np.dot(mass_jac_tmp[comp_idx_1, num_statevars:], c_G)
            for comp_idx_2 in range(comp_idx_1, num_components):
                equilibrium_matrix[num_stable_phases+comp_idx_1, comp_idx_2] += phase_amt[idx] * np.dot(mass_jac_tmp[comp_idx_1, num_statevars:], c_component[comp_idx_2, :])
                equilibrium_matrix[num_stable_phases+comp_idx_2, comp_idx_1] = equilibrium_matrix[num_stable_phases+comp_idx_1, comp_idx_2]
        print('e_matrix', e_matrix)
        print('c_component', c_component)
        print('eq[1,0]', phase_amt[0] * np.dot(mass_jac_tmp[0, num_statevars:], np.dot(e_matrix, mass_jac_tmp[0, num_statevars:])))
        print('eq[2,0]', phase_amt[0] * np.dot(mass_jac_tmp[1, num_statevars:], np.dot(e_matrix, mass_jac_tmp[0, num_statevars:])))
        print('eq[1,1]', phase_amt[0] * np.dot(mass_jac_tmp[0, num_statevars:], np.dot(e_matrix, mass_jac_tmp[1, num_statevars:])))
        print('eq[2,1]', phase_amt[0] * np.dot(mass_jac_tmp[1, num_statevars:], np.dot(e_matrix, mass_jac_tmp[1, num_statevars:])))
        print('rhs[0]', energy_tmp[0])
        print('rhs[1]', phase_amt[0] * np.dot(mass_jac_tmp[0, num_statevars:], c_G))
        print('rhs[2]', phase_amt[0] * np.dot(mass_jac_tmp[1, num_statevars:], c_G))
    print(equilibrium_matrix, equilibrium_rhs)
    equilibrium_soln = np.linalg.solve(equilibrium_matrix, equilibrium_rhs)
    print('soln', equilibrium_soln)
    old_phase_amt = np.array(phase_amt)
    phase_amt += equilibrium_soln[num_components:]
    print(phase_amt, chemical_potentials)
    chemical_potentials[:] = equilibrium_soln[:num_components]

LIQUID 0 [-2.67563749e-13  2.67674771e-13]
LIQUID 0 [-5.55111512e-17  0.00000000e+00]
LIQUID 0 [-5.55111512e-17 -1.11022302e-16]
LIQUID 0 [5.55111512e-17 1.11022302e-16]
LIQUID 0 [-5.55111512e-17 -1.11022302e-16]
LIQUID 0 [5.55111512e-17 1.11022302e-16]
LIQUID 0 [-5.55111512e-17 -1.11022302e-16]
LIQUID 0 [5.55111512e-17 1.11022302e-16]
LIQUID 0 [-5.55111512e-17 -1.11022302e-16]
LIQUID 0 [5.55111512e-17 1.11022302e-16]
e_matrix [[ 8.419027e-05 -8.419027e-05]
 [-8.419027e-05  8.419027e-05]]
c_component [[ 8.419027e-05 -8.419027e-05]
 [-8.419027e-05  8.419027e-05]]
eq[1,0] 8.419027001018017e-05
eq[2,0] -8.419027001018017e-05
eq[1,1] -8.419027001018017e-05
eq[2,1] 8.419027001018017e-05
rhs[0] [-1523.70937183]
rhs[1] 0.17793255068148967
rhs[2] -0.17793255068148967
[[ 3.000000e-01  7.000000e-01  0.000000e+00]
 [ 8.419027e-05 -8.419027e-05  3.000000e-01]
 [-8.419027e-05  8.419027e-05  7.000000e-01]] [-1.52370937e+03  1.77932551e-01 -1.77932551e-01]
soln [  -44.28917919 -2157.74659725     0.  

In [7]:
import numpy as np
a = np.array([[ 3.0000000e-01,  7.0000000e-01],
              [ 7.3160462e-05, -7.3160462e-05]])
b = np.array(([-1.52370937e+03,  0]))
np.linalg.solve(a,b)

array([-1523.70937, -1523.70937])